## 前置作業
在Anaconda Prompt中分別輸入:
- pip install mido
- pip install pygame

## 執行
依序執行所有cell，並打開MidiWrite視窗介面\
打開scores.xlsx檔，開始編寫歌曲
## 工作表數值存放位置
- 歌曲名稱: 工作表名稱
- BPM: A2
- 音軌數: A4
- 演奏樂器: A6、A7、A8
- 音符: B、D、F...
- 音符長度: C、E、G...

## 工作表編寫方式
- 歌曲名稱: 寫入歌曲的名稱，生成的.mid檔會以此命名
- BPM: 每分鐘的拍數，1拍=$\frac{60}{BPM}$秒
- 音軌數: 音樂的音軌數量
- 演奏樂器: 每個音軌要演奏的樂器
- 音符:
    - do: C / c
    - re: D / d
    - mi: E / e
    - fa: F / f
    - so: G / g
    - la: A / a
    - si: B / b
    - 升記號;降記號: #;b
    - 八度音: 0~8
    - 休止符: rest / REST / Rest
- 音符長度: 音符的持續長度

In [15]:
import os
import tkinter as tk
import pygame.midi
from openpyxl import load_workbook
from mido import Message, MidiFile, MidiTrack

PATH = os.path.abspath('.\\mido')
NOTEXLSX = PATH + '\\scores.xlsx'
FONT = 'font.otf'

In [16]:
class Database:
    def __init__(self):
        self.standard = {
            'C': 0,
            'C#': 1,
            'Db': 1,
            'D': 2,
            'D#': 3,
            'Eb': 3,
            'E': 4,
            'F': 5,
            'F#': 6,
            'Gb': 6,
            'G': 7,
            'G#': 8,
            'Ab': 8,
            'A': 9,
            'A#': 10,
            'Bb': 10,
            'B': 11
        }
        self.standard_lowercase = {
            'c': 0,
            'c#': 1,
            'db': 1,
            'd': 2,
            'd#': 3,
            'eb': 3,
            'e': 4,
            'f': 5,
            'f#': 6,
            'gb': 6,
            'g': 7,
            'g#': 8,
            'ab': 8,
            'a': 9,
            'a#': 10,
            'bb': 10,
            'b': 11
        }
        self.standard.update(self.standard_lowercase)
        
        self.rest = ['rest', 'Rest', 'REST']
        
        self.instruments = {
            'Acoustic Grand Piano': 1,
            'Bright Acoustic Piano': 2,
            'Electric Grand Piano': 3,
            'Honky-tonk Piano': 4,
            'Electric Piano 1': 5,
            'Electric Piano 2': 6,
            'Harpsichord': 7,
            'Clavi': 8,
            'Celesta': 9,
            'Glockenspiel': 10,
            'Music Box': 11,
            'Vibraphone': 12,
            'Marimba': 13,
            'Xylophone': 14,
            'Tubular Bells': 15,
            'Dulcimer': 16,
            'Drawbar Organ': 17,
            'Percussive Organ': 18,
            'Rock Organ': 19,
            'Church Organ': 20,
            'Reed Organ': 21,
            'Accordion': 22,
            'Harmonica': 23,
            'Tango Accordion': 24,
            'Acoustic Guitar (nylon)': 25,
            'Acoustic Guitar (steel)': 26,
            'Electric Guitar (jazz)': 27,
            'Electric Guitar (clean)': 28,
            'Electric Guitar (muted)': 29,
            'Overdriven Guitar': 30,
            'Distortion Guitar': 31,
            'Guitar harmonics': 32,
            'Acoustic Bass': 33,
            'Electric Bass (finger)': 34,
            'Electric Bass (pick)': 35,
            'Fretless Bass': 36,
            'Slap Bass 1': 37,
            'Slap Bass 2': 38,
            'Synth Bass 1': 39,
            'Synth Bass 2': 40,
            'Violin': 41,
            'Viola': 42,
            'Cello': 43,
            'Contrabass': 44,
            'Tremolo Strings': 45,
            'Pizzicato Strings': 46,
            'Orchestral Harp': 47,
            'Timpani': 48,
            'String Ensemble 1': 49,
            'String Ensemble 2': 50,
            'SynthStrings 1': 51,
            'SynthStrings 2': 52,
            'Choir Aahs': 53,
            'Voice Oohs': 54,
            'Synth Voice': 55,
            'Orchestra Hit': 56,
            'Trumpet': 57,
            'Trombone': 58,
            'Tuba': 59,
            'Muted Trumpet': 60,
            'French Horn': 61,
            'Brass Section': 62,
            'SynthBrass 1': 63,
            'SynthBrass 2': 64,
            'Soprano Sax': 65,
            'Alto Sax': 66,
            'Tenor Sax': 67,
            'Baritone Sax': 68,
            'Oboe': 69,
            'English Horn': 70,
            'Bassoon': 71,
            'Clarinet': 72,
            'Piccolo': 73,
            'Flute': 74,
            'Recorder': 75,
            'Pan Flute': 76,
            'Blown Bottle': 77,
            'Shakuhachi': 78,
            'Whistle': 79,
            'Ocarina': 80,
            'Lead 1 (square)': 81,
            'Lead 2 (sawtooth)': 82,
            'Lead 3 (calliope)': 83,
            'Lead 4 (chiff)': 84,
            'Lead 5 (charang)': 85,
            'Lead 6 (voice)': 86,
            'Lead 7 (fifths)': 87,
            'Lead 8 (bass + lead)': 88,
            'Pad 1 (new age)': 89,
            'Pad 2 (warm)': 90,
            'Pad 3 (polysynth)': 91,
            'Pad 4 (choir)': 92,
            'Pad 5 (bowed)': 93,
            'Pad 6 (metallic)': 94,
            'Pad 7 (halo)': 95,
            'Pad 8 (sweep)': 96,
            'FX 1 (rain)': 97,
            'FX 2 (soundtrack)': 98,
            'FX 3 (crystal)': 99,
            'FX 4 (atmosphere)': 100,
            'FX 5 (brightness)': 101,
            'FX 6 (goblins)': 102,
            'FX 7 (echoes)': 103,
            'FX 8 (sci-fi)': 104,
            'Sitar': 105,
            'Banjo': 106,
            'Shamisen': 107,
            'Koto': 108,
            'Kalimba': 109,
            'Bag pipe': 110,
            'Fiddle': 111,
            'Shanai': 112,
            'Tinkle Bell': 113,
            'Agogo': 114,
            'Steel Drums': 115,
            'Woodblock': 116,
            'Taiko Drum': 117,
            'Melodic Tom': 118,
            'Synth Drum': 119,
            'Reverse Cymbal': 120,
            'Guitar Fret Noise': 121,
            'Breath Noise': 122,
            'Seashore': 123,
            'Bird Tweet': 124,
            'Telephone Ring': 125,
            'Helicopter': 126,
            'Applause': 127,
            'Gunshot': 128
        }

database = Database()

In [17]:
def note(notes, duration, track, velocity = 100):
    def getNoteNum(note):
        name = note[:-1]
        octave = int(note[-1]) + 1
        return database.standard[name] + 12 * octave
        
    notes = notes.replace(' ', '').split(',')
    notes = [getNoteNum(note) for note in notes]
    
    for note in notes:
        track.append(Message("note_on", note = note, velocity = velocity, time = 0))
        
    for note in notes:
        if notes.index(note) > 0:
            duration = 0
        
        track.append(Message("note_off", note = note, velocity = velocity, time = round(duration)))
            
def rest(duration, track):
    track.append(Message("note_off", time = round(duration)))
    
def track(notesList, durationList, beat_time, instrument, track):
    notes = [note.value for note in notesList if note.value != None]
    duration = [beat_time * float(beat.value) for beat in durationList if beat.value != None]
    
    track.append(Message("program_change", program = instrument))

    for i in range(len(notes)):
        if notes[i] in database.rest:
            rest(duration[i], track)
        else:
            note(notes[i], duration[i], track)
            
    rest(beat_time * 1, track)

In [18]:
def writeMidi(name, sheet, path):
    BPM = int(sheet['A2'].value)
    beat_time = 60 / BPM * 1000
    trackCount = int(sheet['A4'].value)
    
    midi = MidiFile()
    for i in range(trackCount):
        newTrack = MidiTrack()
        midi.tracks.append(newTrack)
        instrument = database.instruments[sheet['A' + str(6 + i)].value]
        track(sheet[chr(ord("B") + i * 2)], sheet[chr(ord("C") + i * 2)], 
              beat_time, instrument, newTrack)
    
    midi.save(path)

In [19]:
def playMidi(file):
    freq = 44100
    bitsize = -16
    channels = 2
    buffer = 1024
    pygame.mixer.init(freq, bitsize, channels, buffer)
    pygame.mixer.music.set_volume(1)
    
    try:
        pygame.mixer.music.load(file)
        pygame.mixer.music.play()
        return True
    except:
        return False

In [20]:
class window:
    def __init__(self):
        self.wb = load_workbook(NOTEXLSX)
        self.sheets = [sheet for sheet in self.wb]
        self.songs = [sheet.title for sheet in self.wb]
        self.filepaths = [PATH + '\music\\' + name + '.mid' for name in self.songs]
        self.songindex = 0

        self.window = tk.Tk()
        self.window.title('MidiWrite')
        self.window.geometry('500x600+1000+300')

        def reload():
            self.wb = load_workbook(NOTEXLSX)
            self.sheets = [sheet for sheet in self.wb]
            self.songs = [sheet.title for sheet in self.wb]
            self.filepaths = [PATH + '\music\\' + name + '.mid' for name in self.songs]
            self.songindex = min(self.songindex, len(self.songs) - 1)
            lab_songname.configure(text = self.songs[self.songindex])
            lab_hint.configure(text = f'讀取完畢\n{self.songs}\n')

        def last():
            self.songindex = (self.songindex - 1 + len(self.songs)) % len(self.songs)
            lab_songname.configure(text = self.songs[self.songindex])
            lab_hint.configure(text = f'上一首歌\n{self.songs[self.songindex]}\n')
            
        def next():
            self.songindex = (self.songindex + 1) % len(self.songs)
            lab_songname.configure(text = self.songs[self.songindex])
            lab_hint.configure(text = f'下一首歌\n{self.songs[self.songindex]}\n')

        def write():
            name = self.songs[self.songindex]
            sheet = self.sheets[self.songindex]
            filepath = self.filepaths[self.songindex]
            writeMidi(name, sheet, filepath)
            lab_hint.configure(text = f'寫入成功\n{name}:\n{filepath}\n')
            
        def play():
            name = self.songs[self.songindex]
            filepath = self.filepaths[self.songindex]
            success = playMidi(filepath)
            if success:
                lab_hint.configure(text = f'播放成功\n{name}:\n{filepath}\n')
            else:
                lab_hint.configure(text = f'播放失敗\n{name}:\n{filepath}\n')
            
        lab_songname = tk.Label(self.window,
            text = self.songs[self.songindex], font = (FONT, 22))
        btn_reload = tk.Button(self.window,
            text = "重新讀取", font = (FONT, 16),
            command = reload)
        btn_last = tk.Button(self.window,
            text = "上一首", font = (FONT, 16),
            command = last)
        btn_next = tk.Button(self.window,
            text = "下一首", font = (FONT, 16),
            command = next)
        btn_write = tk.Button(self.window,
            text = "寫入midi", font = (FONT, 16),
            command = write)
        btn_play = tk.Button(self.window,
            text = "播放midi", font = (FONT, 16),
            command = play)
        lab_hint = tk.Label(self.window,
            text = "", font = (FONT, 22))

        lab_songname.pack()
        btn_reload.pack()
        btn_last.pack()
        btn_next.pack()
        btn_write.pack()
        btn_play.pack()
        lab_hint.pack()
        
        self.window.mainloop()

In [ ]:
window = window()